
# PLP Backend: Query–Response RAG Skeleton (with optional CoT-style reasoning)

This notebook provides a minimal, **fully runnable** backend skeleton for your Personal Learning Portal (PLP) in **Corporate Finance**.  
It implements:
- Lightweight **ingestion → chunking → vector retrieval** (TF–IDF + cosine)
- A simple **answer synthesizer** that cites sources
- Optional, visible **“Reasoning steps” (CoT-style)** for teaching/learning transparency
- Clean Python API: `answer(query, k=4, show_steps=True)`

> No external APIs are required. You can later swap the vectorizer/LLM with your preferred stack (e.g., sentence-transformers + OpenAI) while keeping the same pipeline shape.


In [ ]:

# ==== 0. Imports & Environment ====
import os, math, json, textwrap, uuid
from dataclasses import dataclass, asdict
from typing import List, Dict, Any, Tuple
import numpy as np

try:
    from sklearn.feature_extraction.text import TfidfVectorizer
    from sklearn.metrics.pairwise import cosine_similarity
except Exception as e:
    raise RuntimeError("scikit-learn not found. Please install scikit-learn to run this notebook.") from e



## 1. Toy Learning Corpus (Corporate Finance)
Below we create a **small, local corpus** with metadata so the pipeline runs end-to-end without internet.  
Replace/extend this with your Phase‑3 curated sources (e.g., `data/corpus_sources.json` + actual text).


In [ ]:

# ==== 1. In-memory corpus (replace with your own) ====

toy_corpus = [
    {
        "id": "doc_wacc_01",
        "title": "Estimating Weighted Average Cost of Capital (WACC)",
        "source": "Local notes",
        "url": "local://wacc",
        "text": (
            "The weighted average cost of capital (WACC) is the average rate that a company is expected to pay to "
            "finance its assets, weighted by the proportion of each financing source. WACC = E/V * Re + D/V * Rd * (1 - Tc), "
            "where E is market value of equity, D is market value of debt, V=E+D, Re is cost of equity, Rd is cost of debt, "
            "and Tc is corporate tax rate. WACC is used as a discount rate in NPV analysis when the project's risk is similar "
            "to the firm's average risk profile."
        )
    },
    {
        "id": "doc_npv_01",
        "title": "NPV and IRR Basics",
        "source": "Local notes",
        "url": "local://npv",
        "text": (
            "Net Present Value (NPV) discounts expected cash flows at an appropriate rate to determine project value. "
            "NPV = sum(CFt / (1 + r)^t) - initial_investment. If NPV > 0, accept. The Internal Rate of Return (IRR) is the rate "
            "that sets NPV to zero; it works best for conventional cash flows and when comparing projects of similar scale. "
            "Payback period ignores time value of money and cash flows after cutoff."
        )
    },
    {
        "id": "doc_mm_01",
        "title": "Modigliani–Miller with Taxes (Intuition)",
        "source": "Local notes",
        "url": "local://mm",
        "text": (
            "With corporate taxes, interest is tax-deductible, creating a tax shield that can make some leverage beneficial. "
            "Trade-offs arise due to expected costs of financial distress and agency costs. In frictionless markets without taxes, "
            "capital structure is irrelevant to firm value, but real-world frictions lead to target leverage considerations."
        )
    },
    {
        "id": "doc_div_01",
        "title": "Dividend Policy and Signaling",
        "source": "Local notes",
        "url": "local://dividends",
        "text": (
            "Dividend policy involves choosing between distributing cash to shareholders and reinvesting earnings. "
            "Firms often smooth dividends; changes may signal management’s expectations about sustainable cash flows. "
            "Buybacks offer flexibility compared to regular dividends. In perfect markets, payout policy is irrelevant, "
            "but taxes, transaction costs, and information asymmetry matter in practice."
        )
    },
    {
        "id": "doc_gov_01",
        "title": "Governance and Agency in Corporate Finance",
        "source": "Local notes",
        "url": "local://governance",
        "text": (
            "Agency problems occur when managers’ incentives diverge from shareholders’. Governance mechanisms include boards, "
            "executive compensation, ownership structure, and debt covenants. Transparent disclosure and investor protections "
            "help mitigate agency costs and align decisions with long-term value."
        )
    }
]

len(toy_corpus)



## 2. Preprocessing & Indexing
We perform simple **sentence-level chunking** and build a **TF–IDF index** for retrieval.


In [ ]:

# ==== 2. Chunking ====
import re

def simple_sentence_split(text: str) -> List[str]:
    # naive sentence splitter
    sents = re.split(r'(?<=[.!?])\s+', text.strip())
    return [s for s in sents if s]

@dataclass
class Chunk:
    doc_id: str
    chunk_id: str
    title: str
    url: str
    text: str

def build_chunks(corpus: List[Dict[str, Any]], max_sent_per_chunk: int = 2) -> List[Chunk]:
    chunks = []
    for doc in corpus:
        sents = simple_sentence_split(doc["text"])
        for i in range(0, len(sents), max_sent_per_chunk):
            piece = " ".join(sents[i:i+max_sent_per_chunk])
            chunks.append(
                Chunk(
                    doc_id=doc["id"],
                    chunk_id=f"{doc['id']}::ch{i//max_sent_per_chunk}",
                    title=doc["title"],
                    url=doc["url"],
                    text=piece
                )
            )
    return chunks

chunks = build_chunks(toy_corpus, max_sent_per_chunk=2)
len(chunks), chunks[0]


In [ ]:

# ==== 3. Vector Index (TF–IDF) ====

class TfidfIndex:
    def __init__(self, chunks: List[Chunk]):
        self.chunks = chunks
        self.vectorizer = TfidfVectorizer(ngram_range=(1,2), stop_words='english')
        self.matrix = self.vectorizer.fit_transform([c.text for c in chunks])

    def query(self, q: str, top_k: int = 5) -> List[Tuple[Chunk, float]]:
        qv = self.vectorizer.transform([q])
        sims = cosine_similarity(qv, self.matrix)[0]
        idx = np.argsort(-sims)[:top_k]
        return [(self.chunks[i], float(sims[i])) for i in idx]

index = TfidfIndex(chunks)
results = index.query("How do I compute WACC and when to use it?", top_k=3)
results



## 3. CoT-style Reasoning & Synthesis
We generate **visible reasoning steps** (optional) and a final answer that **cites** the retrieved chunks.


In [ ]:

# ==== 4. Reasoner & Synthesizer ====

def cot_plan(query: str) -> List[str]:
    return [
        "1) Parse the question and identify key finance concepts.",
        "2) Retrieve definitions/formulas and usage criteria from the corpus.",
        "3) Cross-check retrieved evidence and select the most relevant chunks.",
        "4) Synthesize an answer with definitions, formula(s), and when-to-use guidance.",
        "5) List concise citations to source chunks."
    ]

def synthesize_answer(query: str, retrieved: List[Tuple[Chunk, float]], show_steps: bool=True) -> Dict[str, Any]:
    # Build a short, grounded answer from retrieved chunks
    bullet_evidence = []
    for ch, score in retrieved:
        snippet = textwrap.shorten(ch.text, width=220, placeholder="…")
        bullet_evidence.append(f"- [{ch.doc_id}] {snippet} (score={score:.3f})")

    # Very simple "generation": stitch relevant pieces
    # In practice, replace with your LLM call and keep citations.
    top_texts = " ".join([ch.text for ch, _ in retrieved])

    # Heuristic: if WACC mentioned, include formula; if NPV/IRR mentioned, add usage
    answer_lines = []
    qt = query.lower()
    if "wacc" in qt or "cost of capital" in qt:
        answer_lines.append(
            "Weighted Average Cost of Capital (WACC) is the firm’s blended discount rate: "
            "WACC = E/V·Re + D/V·Rd·(1−Tc). Use it as the discount rate for projects with risk similar to the firm’s core assets."
        )
    if "npv" in qt or "irr" in qt:
        answer_lines.append(
            "NPV discounts expected cash flows at an appropriate rate; accept if NPV>0. "
            "IRR is the rate that sets NPV=0; works best for conventional cash flows and comparable scale projects."
        )
    if not answer_lines:
        # default summary from retrieved
        answer_lines.append(textwrap.shorten(top_texts, width=500, placeholder="…"))

    citations = [{"doc_id": ch.doc_id, "chunk_id": ch.chunk_id, "url": ch.url} for ch, _ in retrieved]

    out = {
        "query": query,
        "answer": " ".join(answer_lines),
        "citations": citations
    }
    if show_steps:
        out["reasoning_steps"] = cot_plan(query) + bullet_evidence
    return out



## 4. Public API
Use `answer(query, k=4, show_steps=True)` to run the full pipeline.


In [ ]:

# ==== 5. Public API ====

def answer(query: str, k: int = 4, show_steps: bool=True) -> Dict[str, Any]:
    retrieved = index.query(query, top_k=k)
    return synthesize_answer(query, retrieved, show_steps=show_steps)

# Quick smoke tests
demo1 = answer("How do I compute WACC and when should I use it as a discount rate?")
demo2 = answer("Compare NPV and IRR and mention a limitation of payback.", show_steps=False)

demo1, demo2



## 5. (Optional) Extend: Ingest Your Own Corpus
Use `ingest_documents()` to add your texts (e.g., from SEC filings, textbooks).  
Rebuild the index with `rebuild_index()`.


In [ ]:

# ==== 6. Ingestion Helpers ====

def ingest_documents(docs: List[Dict[str, str]]):
    """docs: list of dicts with keys: id, title, url, text"""
    ids = {d["id"] for d in toy_corpus}
    for d in docs:
        if d["id"] in ids:
            raise ValueError(f"Duplicate id: {d['id']}")
        toy_corpus.append(d)

def rebuild_index(max_sent_per_chunk: int = 2):
    global chunks, index
    chunks = build_chunks(toy_corpus, max_sent_per_chunk=max_sent_per_chunk)
    index = TfidfIndex(chunks)

# Example (commented):
# ingest_documents([
#     {"id":"doc_custom_01","title":"My notes on CAPM","url":"local://capm","text":"Cost of equity can be estimated via CAPM: Re = Rf + beta*(ERP)."},
# ])
# rebuild_index()
# answer("How to estimate cost of equity using CAPM?")



## 6. (Optional) Plug-in Points
- **Embeddings**: swap TF–IDF with sentence embeddings (e.g., `sentence-transformers`)  
- **Reranking**: add a cross-encoder or a BM25 first-stage + cross-encoder reranker  
- **LLM**: replace `synthesize_answer()` with an LLM call, keep the `citations` contract  
- **Eval**: log queries/answers and compute groundedness/faithfulness with RAGAs/ARES later



## 7. Minimal CLI
Uncomment and run the cell to ask interactive questions inside the notebook.


In [ ]:

# ==== 7. Minimal CLI (optional) ====
# while True:
#     q = input("Ask a corporate-finance question (or 'quit'): ").strip()
#     if q.lower() in {"quit", "exit"}:
#         break
#     resp = answer(q, k=4, show_steps=True)
#     print("\nAnswer:", resp["answer"])
#     print("Citations:", json.dumps(resp["citations"], indent=2))
#     print("Reasoning:", *resp.get("reasoning_steps", []), sep="\n- ")
